In [ ]:
from preprocess import Preprocess
import pandas as pd

**NOTA**: boundary, buildings e samples devono esser gia' stati proiettati in un CRS metrico. Usare epsg=2154 (CRS metrico) per la Francia!

#### Instantiating the Preprocess class...OK

In [ ]:
preprocessor = Preprocess("Paris")

#### Processing the raw buildings and POIs downloaded from Geofabrik...OK

In [ ]:
building, poi = preprocessor.get_building_and_poi()

#### Performing Poisson Disk Sampling...OK

In [ ]:
radius = 50
random_point = preprocessor.poisson_disk_sampling(building, poi, radius)

#### Rasterize buildings

**TODO**: il metodo di rasterizzazione degli autori richiede un sacco di memoria, e.g., 155 giga per allocare i raster 224x224 di 3.3 milioni di building in Parigi. Cercare di renderlo memory efficient (e.g., batch processing?).

#### Partition city data by road network...OK

**TODO**: gli autori segmentano una citta' tramite le traffic analysis zones. Problema: non ci sono shapefiles disponibili per Parigi. Per ora usiamo IRIS al posto delle TAZ, hanno densita' comparabile ed i limiti delle celle IRIS seguono la road network (analogamente alle celle TAZ).

In [ ]:
preprocessor.partition(building, poi, random_point, radius)
print(f'Random Points: {len(random_point)}')